In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Conv2D, Flatten, Reshape, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns

#Chargement des donnees CIFAR-10

(x_train, _), (x_test,_ ) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

#Parametres
input_shape = (32,32,3)
latent_dim = 16
intermediate_dim = 512
beta= 1.0

#Encoder
inputs = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Flatten()(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

def sampling(args):
  z_mean, z_log_var = args
  epsillon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
  return z_mean + K.exp(z_log_var / 2) * epsillon

z = Lambda(sampling)([z_mean, z_log_var])

#Decoder
decoder_inputs = Input(shape=(latent_dim,))
x= Dense(8*8*64, activation='relu')(decoder_inputs)
x = Reshape((8,8,64))(x)
x = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu')(x)
x_decoded = Conv2DTranspose(3, (3,3), activation='sigmoid', padding='same')(x)

decoder = Model(decoder_inputs, x_decoded)
x_decoded_mean = decoder(z)

class VAELossLayer(tf.keras.layers.Layer):
  def __init__(self, beta=0.1, **kwargs):
    super(VAELossLayer, self).__init__(**kwargs)
    self.beta = beta

  def call(self, inputs):
    x = inputs[0]
    x_decoded_mean = inputs[1]
    z_mean = inputs[2]
    z_log_var = inputs[3]
    reconstruction_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(x, x_decoded_mean), axis=(1,2)
    )
    kl_loss = -0.5 * tf.reduce_sum(
        1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1
    )
    self.add_loss(tf.reduce_mean(reconstruction_loss + self.beta * kl_loss))
    return x_decoded_mean

vae_output = VAELossLayer(beta=beta)([inputs, x_decoded_mean, z_mean, z_log_var])

vae = Model(inputs, vae_output)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

vae.fit(x_train, x_train, epochs=10, batch_size=1024, validation_data=(x_test, x_test))



Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - loss: 705.7597 - val_loss: 689.0307
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 227s 5s/step - loss: 683.2609 - val_loss: 672.6968
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 226s 4s/step - loss: 668.2838 - val_loss: 655.4686
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - loss: 651.2939 - val_loss: 643.3610
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - loss: 641.8433 - val_loss: 640.1822
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - loss: 637.6491 - val_loss: 635.5073
Epoch 7/10
37/49 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - loss: 634.4376

KeyboardInterrupt: 

In [ ]:
#Reconstructions
def plot_reconstructions (data, model, n=10):
  samples= data[:n]
  reconstructions= model.predict(samples)
  plt.figure(figsize=(20, 4))
  for i in range(n):
    # Image originale
    plt.subplot(2, n, i + 1)
    plt.imshow(samples[i])
    plt.title("Original")
    plt.axis("off")

    # Reconstruction
    plt.subplot(2, n, i+1 + n)
    plt.imshow(reconstructions [i])
    plt.title("Reconstructed")
    plt.axis("off")
  plt.tight_layout()
  plt.show()
plot_reconstructions (x_test, vae)

In [ ]:
# Visualisation de l'espace latent
encoder= Model (inputs, z_mean)
latent_space =encoder.predict(x_test)
z_tsne = TSNE(n_components=2).fit_transform (latent_space)
plt.figure(figsize=(10, 8))
sns.scatterplot(x=z_tsne[:, 0], y=z_tsne[:, 1], s=10, palette="tab10")
plt.title("Clusters dans l'Espace Latent")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()
# Génération d'images
generator_input =Input(shape=(latent_dim,))
x = Dense (8*8*64, activation="relu") (generator_input)
x = Reshape((8, 8, 64)) (x)
x = Conv2DTranspose (64, (3, 3), activation="relu", strides=(2, 2), padding="same")(x)
x = Conv2DTranspose (32, (3, 3), activation="relu", strides=(2, 2), padding="same")(x)
generated_output = Conv2DTranspose (3, (3, 3), activation="sigmoid", padding="same")(x)
generator = Model (generator_input, generated_output)
def generate_images (generator, n=10):
  grid_x = np.linspace (-3, 3, n)
  grid_y = np.linspace(-3, 3, n)
  figure= np.zeros((32 * n, 32 * n, 3))
  for i, xi in enumerate (grid_x):
    for j, yi in enumerate (grid_y):
      z_sample = np.array ([[xi, yi]])
      x_decoded = generator.predict(z_sample)
      digit = x_decoded[0].reshape((32, 32, 3))